In [56]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random

In [57]:
import torchvision
import torchvision.transforms as transforms

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

Device available now: cuda


In [59]:
image_transforms = transforms.Compose(
         [transforms.ToTensor(),
         transforms.CenterCrop(28),
         transforms.Normalize((0.5), (0.5))]
)


In [60]:
batchsize = 256
rowsize = 28
colsize = 28
inchannels = 1
kernelsize = 5
stride = 1
padding = 0
poolsize = 3
fc_size1 = 384
fc_size2 = 192
no_classes = 10
epochs = 100


In [61]:
train = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=image_transforms)
test = torchvision.datasets.MNIST(root='./', train=False, transform=image_transforms)


In [62]:
traindata = torch.utils.data.DataLoader(train,batch_size=batchsize, shuffle=True, num_workers=2,pin_memory=True)
testdata = torch.utils.data.DataLoader(test,batch_size=batchsize, shuffle=True, num_workers=2,pin_memory=True)

In [63]:
class SimpleAlex(nn.Module):
  def __init__(self,inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes):
    super(SimpleAlex, self).__init__()
    self.conv1 = nn.Conv2d(inchannels,24,kernelsize,stride,padding)
    self.pool1 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm1 = nn.BatchNorm2d(24)
    self.conv2 = nn.Conv2d(24,48,kernelsize,stride,padding)
    self.pool2 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm2 = nn.BatchNorm2d(48)
    self.fc1 = nn.Linear(432,fc_size1)
    self.fc2 = nn.Linear(fc_size1,fc_size2)
    self.fc3 = nn.Linear(fc_size2,no_classes) 

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = self.BatchNorm1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = self.BatchNorm2(x)
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return x

# class SimpleAlex(nn.Module):
#     def __init__(self, num_classes):
#         super(SimpleAlex, self).__init__()
#         self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
#         self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
#         self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
#         self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
#         self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.fc1 = nn.Linear(1600, 128)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(128, num_classes)
#     def forward(self, x):
#         out = self.conv_layer1(x)
#         out = self.conv_layer2(out)
#         out = self.max_pool1(out)
        
#         out = self.conv_layer3(out)
#         out = self.conv_layer4(out)
#         out = self.max_pool2(out)
                
#         out = out.reshape(out.size(0), -1)
        
#         out = self.fc1(out)
#         out = self.relu1(out)
#         out = self.fc2(out)
#         return out

In [64]:
model = SimpleAlex(inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes).to(device)
# model = SimpleAlex(no_classes).to(device)

In [65]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss().to(device)

In [66]:
# def training(model,traindata,testdata):
def train_test(epochs):
 list_acc = 0
 for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  model.train()
  for i,data in enumerate(traindata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data.cuda())
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    
  list_acc += num_correct/tot
 

 num_correct = 0
 tot = 0
 model.eval()
 for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
 return (num_correct)/tot , list_acc/epochs

    

In [68]:
list_x = [i+1 for i in range(30)]
list_test = []
list_train = []
for i in list_x:
  
  a,b = train_test(i)
  print(a,b)
  list_test.append(a)
  list_train.append(b)



0.8296 0.5960666666666666
0.9716 0.8772583333333334


KeyboardInterrupt: ignored

In [69]:
image_transforms_c = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])
])

In [70]:
train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms_c)
test = torchvision.datasets.CIFAR10(root='./', train=False, transform=image_transforms_c)

Files already downloaded and verified


In [72]:
def train_test1(epochs):
 list_acc = 0
 for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  model.train()
  for i,data in enumerate(traindata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data.cuda())
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    
  list_acc += num_correct/tot
 

 num_correct = 0
 tot = 0
 model.eval()
 for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
 return (num_correct)/tot , list_acc/epochs

In [75]:
1list_x = [i+1 for i in range(30)]
list_test = []
list_train = []
for i in list_x:
  
  a,b = train_test1(i)
  print(a,b)
  list_test.append(a)
  list_train.append(b)

SyntaxError: ignored